# Generate Poster Figures

In [5]:
import os
import copy

import os
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import peyes

pio.renderers.default = "browser"

In [6]:
RAW_FIGURES_DIR = r'S:\Lab-Shared\Experiments\pEYES\output\article_results'
OUTPUT_DIR = r'S:\Lab-Shared\Experiments\pEYES\article\poster\figures'
DATASET = "lund2013"

TITLE_FONT = dict(family="Calibri", size=100, color="black")
AXIS_LABEL_FONT = dict(family="Calibri", size=90, color="black")
AXIS_TICK_FONT = dict(family="Calibri", size=70, color="black")

In [7]:
def load_interim_figure(
        dataset_name: str, fig_num: int, panel_id: str, fig_name: str, is_supp: bool
) -> go.Figure:
    sup_dir = os.path.join(RAW_FIGURES_DIR, dataset_name, 'figures')
    if not os.path.isdir(sup_dir):
        raise NotADirectoryError(f"Directory '{sup_dir}' does not exist.")
    fig_dir = os.path.join(sup_dir, f"{'supp-' if is_supp else ''}fig{fig_num}")
    if not os.path.isdir(fig_dir):
        raise NotADirectoryError(f"Directory '{fig_dir}' does not exist.")
    full_path = os.path.join(fig_dir, f"{panel_id}_{fig_name}.json")
    if not os.path.isfile(full_path):
        raise FileNotFoundError(f"File '{full_path}' does not exist.")
    return pio.from_json(open(full_path, 'r').read())

## Fig 1: Overview of pEYES

In [8]:
heatmap = load_interim_figure(DATASET, 1, "A", "heatmap", False)
pixels = load_interim_figure(DATASET, 1, "C", "pixels-over-time", False)
scarfs = load_interim_figure(DATASET, 1, "D", "scarfplots", False)

In [9]:
fig1 = make_subplots(
    rows=2, cols=2, shared_xaxes=True, shared_yaxes=False,
    vertical_spacing=0.05, horizontal_spacing=0.025,
    specs=[[{"type": "scatter"}, {"type": "image", "rowspan": 2}], [{"type": "heatmap"}, None]],
)

for tr in pixels.data:
    if tr["name"] == 'v':
        continue
    tr["line_width"] = 10
    fig1.add_trace(tr, row=1, col=1)
for tr in scarfs.data:
    new_tr = copy.deepcopy(tr)
    new_tr["showscale"] = False
    fig1.add_trace(new_tr, row=2, col=1)
heatmap.for_each_trace(lambda trace: fig1.add_trace(trace, row=1, col=2))

fig1.update_layout(
    width=5000, height=2000, font_family="Calibri",
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    legend=dict(
        orientation="h", yanchor="top", y=1.01, xanchor="left", x=0.01, bgcolor='rgba(0, 0, 0, 0)', font=AXIS_TICK_FONT, itemwidth=75,
    ),
    margin=dict(l=0, r=0, t=0, b=0),

    # line plot axes
    xaxis=dict(showticklabels=False, showgrid=False,),
    yaxis=dict(
        title=dict(text="Gaze Location (px)", font=AXIS_LABEL_FONT, standoff=8),
        showgrid=True, gridcolor='lightgray', gridwidth=5,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,
        rangemode="tozero", showticklabels=True, tickfont=AXIS_TICK_FONT,
    ),
    # scarfplot axes
    xaxis3=dict(
        title=dict(text="Time (sample)", font=AXIS_LABEL_FONT, standoff=4),
        showgrid=False,
        tickangle=30, tickmode='array', tickfont=AXIS_TICK_FONT,
        tickvals=np.arange(0, 10001, 1000), ticktext=[f"{val//1000}k" for val in np.arange(0, 10001, 1000)],
    ),
    yaxis3=dict(
        title=dict(text="Annotator", font=AXIS_LABEL_FONT, standoff=4), showgrid=False, tickangle=0,
        tickmode='array', tickvals=scarfs.layout['yaxis']['tickvals'],
        ticktext=[f"{txt}\t" for txt in scarfs.layout['yaxis']['ticktext']],
        tickfont=AXIS_TICK_FONT,
    ),
    # heatmap axes - remove ticks and labels
    xaxis2=dict(showticklabels=False, showgrid=False), yaxis2=dict(showticklabels=False, showgrid=False),
)

fig1.write_image(os.path.join(OUTPUT_DIR, "fig1.png"), scale=3)
fig1.show()

## Fig 2: Detector Agreement with GT
### Measurement: `Cohen's Kappa`

In [10]:
performances = load_interim_figure(DATASET, 3, "", "sample-global-metrics", False)

fig2 = go.Figure()
for tr in performances.data:
    if tr["scalegroup"] != "cohen's_kappa":
        continue
    new_tr = copy.deepcopy(tr)
    new_tr["showlegend"] = False
    new_tr["width"] = 0.95

    if new_tr["x0"] == "Other GT":
        new_tr["x0"] = new_tr["name"] = new_tr["legendgroup"] = "2<sup>nd</sup> GT"
    elif new_tr["x0"].startswith("i"):
        new_tr["x0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
    elif new_tr["x0"] == "remodnav":
        new_tr["x0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
    else:
        new_tr["x0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
    fig2.add_trace(new_tr)

fig2.update_layout(
    width=3600, height=2000, font_family="Calibri",
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    title=dict(
        text="<b>Detector Agreement with GT</b>", font=TITLE_FONT,
        y=1, yanchor="top", x=0.5, xanchor="center"
    ),
    yaxis=dict(
        title=dict(text="<b>Cohen's Kappa</b>", font=AXIS_LABEL_FONT, standoff=45),
        showgrid=True, gridcolor='lightgray', gridwidth=5,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,
        showticklabels=True, tickmode='array', tickvals=np.arange(0.1, 1.0, 0.2), tickfont=AXIS_TICK_FONT,
    ),
    xaxis=dict(
        title=dict(text="<b>Detector</b>", font=AXIS_LABEL_FONT, standoff=25),
        tickfont=AXIS_TICK_FONT,
    ),
)

fig2.write_image(os.path.join(OUTPUT_DIR, "fig2.png"), scale=3)
fig2.show()

## Fig 3: Sensitivity
### Measurement: $d'$ for increasing $\Delta t$

In [11]:
fix_fig = load_interim_figure(DATASET, 6, "", "fixations-discrimination_multi_threshold-d_prime", True)
sac_fig = load_interim_figure(DATASET, 7, "", "saccades-discrimination_multi_threshold-d_prime", True)

In [12]:
fig3 = make_subplots(
    rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.025, horizontal_spacing=0.025,
    column_titles=["Fixations", "Saccades"],
)

for tr in fix_fig.data:
    if tr["yaxis"] in {"y", "y3"}:
        new_tr = copy.deepcopy(tr)
        new_tr["line_width"] = 10
        new_tr["error_y"]["thickness"] = 8
        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = "2<sup>nd</sup> GT"
            new_tr["marker"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()
        fig3.add_trace(new_tr, col=1, row=1 if tr["yaxis"] == "y" else 2)

for tr in sac_fig.data:
    if tr["yaxis"] in {"y", "y3"}:
        new_tr = copy.deepcopy(tr)
        new_tr["line_width"] = 10
        new_tr["error_y"]["thickness"] = 8
        new_tr["showlegend"] = False
        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = "2<sup>nd</sup> GT"
            new_tr["marker"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()
        fig3.add_trace(new_tr, col=2, row=1 if tr["yaxis"] == "y" else 2)

fig3.update_layout(
    width=6000, height=2000, font_family="Calibri",
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    # margin=dict(l=0, r=0, t=20, b=0),
    title=dict(
        text="<b>Sensitivity</b>", font=TITLE_FONT,
        y=1, yanchor="top", x=0.5, xanchor="center"
    ),

    xaxis3=dict(
        title=dict(text="<b>$\\Delta t$</b>", font=AXIS_LABEL_FONT, standoff=0), tickfont=AXIS_TICK_FONT,
    ),
    xaxis4=dict(
        title=dict(text="<b>$\\Delta t$</b>", font=AXIS_LABEL_FONT, standoff=0), tickfont=AXIS_TICK_FONT,
    ),

    yaxis=dict(
        title=dict(text="<b>Onset d'", font=AXIS_LABEL_FONT),
        showgrid=True, gridcolor='lightgray', gridwidth=5,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,
        tickfont=AXIS_TICK_FONT,
    ),
    yaxis3=dict(
        title=dict(text="<b>Offset d'</b>", font=AXIS_LABEL_FONT),
        showgrid=True, gridcolor='lightgray', gridwidth=5,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,
        tickfont=AXIS_TICK_FONT,
    ),
    yaxis2=dict(showgrid=True, gridcolor='lightgray', gridwidth=5, zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,),
    yaxis4=dict(showgrid=True, gridcolor='lightgray', gridwidth=5, zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,),

    legend=dict(
        orientation="h", bgcolor='rgba(0, 0, 0, 0)',
        yanchor="top", y=-0.05, xanchor="center", x=0.5,
        font=AXIS_TICK_FONT, itemwidth=120,
    ),
)
fig3.for_each_annotation(lambda ann: ann.update(font=AXIS_LABEL_FONT, y=0.98))

fig3.write_image(os.path.join(OUTPUT_DIR, "fig3.png"), scale=3)
fig3

## Fig 4: Detection Easiness
### Comparing Onset vs. Offset Detection (across all detectors)

In [13]:
barplots = load_interim_figure(DATASET, 6, "RA", "onset-offset-comparison", False)

fig4 = go.Figure()
for tr in barplots.data:
    if tr["xaxis"] == "x":
        fig4.add_trace(tr)

fig4.update_layout(
    width=3000, height=1500, font_family="Calibri",
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    # margin=dict(l=0, r=0, t=20, b=0),
    title=dict(
        text="<b>Detection Sensitivity across Detectors</b>", font=TITLE_FONT,
        y=0.98, yanchor="top", x=0.5, xanchor="center"
    ),

    xaxis=dict(
        title=dict(text="<b>Event Type</b>", font=AXIS_LABEL_FONT), tickfont=AXIS_TICK_FONT,
    ),
    yaxis=dict(
        title=dict(text="<b>Mean d'", font=AXIS_LABEL_FONT, standoff=25),
        showgrid=True, gridcolor='lightgray', gridwidth=5,
        zeroline=True, zerolinecolor='lightgray', zerolinewidth=5,
        tickfont=AXIS_TICK_FONT,
    ),

    legend=dict(orientation="h", yanchor="top", y=0.95,  xanchor="left", x=0, font=AXIS_TICK_FONT, itemwidth=80),
)

fig4.write_image(os.path.join(OUTPUT_DIR, "fig4.png"), scale=3)
fig4